In [2]:
from pathlib import Path

In [3]:
from docling.document_converter import DocumentConverter
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

c:\Users\ashis\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings

In [5]:
from langchain_ollama import ChatOllama
local_llm = "llama3.2:3b-instruct-fp16"
llm = ChatOllama(model=local_llm,temperature=0)
llm_json_mode = ChatOllama(model=local_llm,temperature=0,format="json")


In [6]:
source = "ambuja test-rotated.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)

# output: ## Docling Technical Report [...]"

c:\Users\ashis\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\cpp_extension.py:384: UserWarning: Error checking compiler version for cl: [WinError 2] The system cannot find the file specified
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')
Could not load the custom kernel for multi-scale deformable attention: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom ker

In [7]:
output_dir = Path("scratch")
with (output_dir / f"markdown.md").open("w", encoding="utf-8") as fp:
        fp.write(result.document.export_to_markdown())
document_path = Path("scratch/markdown.md")

In [8]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
docs = text_splitter.split_documents(loaded_documents)


359

In [12]:
vectorstore = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5",inference_mode="local")
)


In [13]:
retriever = vectorstore.as_retriever(k=3)

In [16]:
from langchain_core.messages import HumanMessage,SystemMessage
##Generating
rag_prompt = """You are an assistant for question-answering tasks.

Here is the context to use to answer the question:

{context}

Think Carefully about the above context.

Now, review the user question:

{question}

Provide an answer to this question using only the above context.

Use three sentences maximum and keep the answer concise.

Answer:"""

##post-prcessing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

question = "How many independent directors are in the board?"
#Test
docs_ret = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt,question=question)
generation = llm.invoke([HumanMessage(content = rag_prompt_formatted)])
print(generation.content)


The company's total revenue for FY 2016-17 and FY 2019-20 was not disclosed. The company has made allotment of 26,54,47,491 equity shares to Harmonia Trade and Investment Limited at a premium of ' 416.87 per share, increasing the promoter group's shareholding from 66.70% to 70.30%. The company has also entered into a definitive agreement for acquisition of a cement grinding unit in Tuticorin, Tamil Nadu on slump sale basis at a total value of ' 413.75 Crores.
